##### El siguiente fragmento permite ocultar las celdas que contienen código

In [45]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

def all_cells():    
    return HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    The raw code for this IPython notebook is by default hidden for easier reading.
    To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

all_cells()

<a id='inicio'></a>
# Proyecto 2: Razonamiento basado en incerteza

## Integrantes: 

* Daniel Julián Rojas Cruz
* Johan Tanta Villanueva

En primer lugar, se muestra la documentación de las funciones que se implementaron para poder llegar a cabo el Proyecto 2. Si desea dirigirse a la presentación del proyecto, haga click [aqui](#presentacion).

Para el correcto funcionamiento del proyecto es necesario importar las siguientes librearias:
* **pandas**: Esta librería es útil para poder trabajar con un dataframe
* **pandas**: Nos ayuda a realizar los histogramas
* **math, random y copy**: Librerías útiles para llevar a cabo el correcto funcionamiento del programa

In [46]:
hide_toggle(True)

In [47]:
import pandas as pd
import numpy as np
import math
import random
import copy

<a id='funciones_iniciales'></a>
Se implementó las siguientes funciones que serán de gran ayuda en todo el proyecto:
* **leer_dataset**: Recibe como parámetro la ruta del archivo .csv y retorna un dataframe de la librería **pandas**.
* **cardinalidad**: Retorna la lista de valores únicos de una columna de un dataframe por lo que recibe como parámetro el dataframe y el nombre de la columna.
* **count**: retorna el número de instancias de un dataframe que satisface una determinada condición.

In [48]:
hide_toggle(for_next=True)

In [49]:
# Esta función recive el nombre del archivo CSV y devuelve un dataframe de Pandas
def leer_dataset(nom_archivo):
    return pd.read_csv(nom_archivo)
# Retorna la lista de valores únicos de una columna de un dataframe
def cardinalidad(df, X):
    return df[X].unique()
#retorna el número de instancias de un dataframe que satisface una determinada condición.
def count(dataframe, index_list, values_list):
    resultado = dataframe
    for i in range(len(index_list)):
        #Se obtiene el nombre de la columna y el valor 
        index = index_list[i]
        value = values_list[i]

        #Se filtra las filas que cumplen con la condicion de esa columna 
        resultado = resultado[resultado[index] == value]
    #Devuelve el numero de instancias
    c =resultado.shape[0]
    return c

<a id='estimar_marginal'></a>

Se implementa la función **estimar_marginal** y **estimar_conjunta** la cual calcula la distribuición con el hiperparámetro de Dirichlet. Estas funciones devuelven un factor.

In [50]:
hide_toggle(True)

In [51]:
# Esta funcióm devuelve un reporte del factor que contiene a la variable X
def estimar_marginal(df,X, alpha):
    card = cardinalidad(df,X) # lista de valores únicos
    
    nfilas = df.shape[0]
    
    probList = []
    for valor in card:
        m = count(df,[X],[valor])
        p = (m + alpha)/(nfilas + len(card)*alpha)
        probList.append(p)
    probColName = "P"
    columns = {X:card, probColName: probList}
    
    return pd.DataFrame(columns)

def estimar_conjunta(df,variables,alpha):
    card = {}
    nfilas = 1
    
    ## filas en el df
    n = df.shape[0]
    
    for var in variables:
        card[var] = cardinalidad(df,var)
        nfilas = nfilas*len(card[var])
        
    columns = {}
    
    n_rep = 1
    for var, cardinal in card.items():
        columns[var] = [ cardinal[int(i/n_rep)%len(cardinal)] for i in range(nfilas) ]
        n_rep = n_rep * len(cardinal)

    result = pd.DataFrame(columns)
    
    probList = []
    
    #Iteramos por cada fila del dataframe y hallamos la probabilidad condicional
    for i in result.index:
        valores = []
        for var in variables:
            valores.append(result[var][i])
      
        n_intersection = count(df,variables,valores)
    
        p = (n_intersection + alpha)/ ( n + n_rep*alpha)
        
        probList.append(p)
    
    result['P'] = probList
    return result

<a id='estimar_condicional'></a>

Se implementa la función **estimar_condicional** la cual calcula la distribuición condicional con el hiperparámetro de Dirichlet. Antes de retornar el factor, se realiza una normalización.

In [52]:
hide_toggle(True)

In [53]:
def estimar_condicional(df, X, Y, alpha):
    # hallar el número total de filas 
    nFilasResult = 1
    nfilas = df.shape[0]
    cardTarget = cardinalidad(df,X)
    nFilasResult = nFilasResult*len(cardTarget)
    
    cardY = {}
    for var in Y:
        cardY[var] = cardinalidad(df,var)
        nFilasResult = nFilasResult*len(cardY[var])
    
    columns = {}
    
    n_rep = 1
    
    colTarget = [cardTarget[i%len(cardTarget)] for i in range(nFilasResult)]
    
    columns[X] = colTarget
    n_rep = n_rep * len(cardTarget)
    
    for var, card in cardY.items():
        columns[var] = [ card[int(i/n_rep)%len(card)] for i in range(nFilasResult) ]
        
        n_rep = n_rep * len(card)

    
    result = pd.DataFrame(columns)
    probList = []
    
    prodCardY = 1
    for var in Y:
        prodCardY = prodCardY* len(cardY[var])
        
    #Iteramos por cada fila del dataframe y hallamos la probabilidad condicional
    for i in result.index:
        valTarget = result[X][i]
    
        valPadres = []
        for var in Y:
            valPadres.append(result[var][i])
      
        totalVar = [i for i in Y]
        totalVar.append(X)
        totalVal = valPadres
        totalVal.append(valTarget)
        
        
        prob_X_Y = (count(df,totalVar,totalVal) + alpha) / (nfilas + prodCardY*len(cardTarget)*alpha) 
        
        prob_Y = (count(df,Y,valPadres) + alpha) / (nfilas + prodCardY*alpha)
        p = prob_X_Y/prob_Y
        probList.append(p)
    
    # normalizamos las probabilidades
    for i in range(int (nFilasResult / len(cardTarget))):
        suma = 0
        for j in range(len(cardTarget)):
            suma = suma + probList[i*len(cardTarget) + j]
        
        for j in range(len(cardTarget)):
            probList[i*len(cardTarget) + j] = probList[i*len(cardTarget) + j] / suma 
    
    result['P'] = probList
    return result

<a id='preprocessor'></a>
## Definición e implementación de la clase Preprocessor
Esta clase recibe como parámetros en su constructor el dataframe de la librería **pandas** y el hiperparámetro. La utilidad de esta clase es para poder poder procesar las distribuiciones tanto marginal, conjunta y condicional. En cuanto a la distribuición condicional solo se procesa las permutaciones de las variables del dataframe de tamaño dos. 

In [54]:
hide_toggle(True)

In [55]:
class Preprocessor:
    def __init__(self, df ,alpha):
        self.variables = self.get_vars(df)
        self.alpha = alpha
        self.df = df
        self.card = self.get_cards(df)
        self.marg_cond = self.preprocess(df)
        self.conj = self.calcular_conjunta(df)
        self.instancias = len(df.index)
    
    def get_cards(self, df):
        nodes = self.variables
        card = {}
        for value in nodes:
            card[value] = df[value].unique()
        return card
    
    def get_vars(self,df):
        variables = []
        for i in df:
            variables.append(i)
        
        return variables
    
    def calcular_conjunta(self,df):
        prob = {}
        
        #calcular las conjunta de cada par de variables diferentes
        for variable_1 in self.variables:
            for variable_2 in self.variables:
                if (variable_1 == variable_2):
                    continue
                prob[tuple([variable_1, variable_2])] = estimar_conjunta(df,[variable_1,variable_2], self.alpha)
        return prob
    
    #preprocesa las marginales y las condicionales para cada para de variables 
    def preprocess(self,df):
        prob = {}
        
        #calcular las marginales de todas la variables
        for variable in self.variables:
            prob[tuple(variable)] = estimar_marginal(df,variable,self.alpha)
        
        #calcular las condicionales de cada par de variables diferentes
        for variable_1 in self.variables:
            for variable_2 in self.variables:
                if (variable_1 == variable_2):
                    continue
                prob[tuple([variable_1, variable_2])] = estimar_condicional(df,variable_1, [variable_2], self.alpha)
        return prob
    
    
    def get_preprocess_marginal(self, variable, valor):
        resultado = self.marg_cond[(variable,)]
        resultado = resultado[resultado[variable]==valor]
        return list(resultado['P'])[0]
    
    def get_preprocess_conjunta(self, variables, valores):
        resultado = self.conj[variables]
        for i in range(2):
            index = variables[i]
            value = valores[i]
            #filtrar la columna 
            resultado = resultado[resultado[index] == value]
            
        return list(resultado['P'])[0]
    
    def get_preprocess_condicional(self, factor, valores):
        resultado = self.marg_cond[factor]
        for i in range(2):
            index = factor[i]
            value = valores[i]
            #filtrar la columna 
            resultado = resultado[resultado[index] == value]
        return list(resultado['P'])[0]

<a id='red_bayesiana'></a>
## Definición e implementación de la clase Red_Bayesiana
Uno de los parámetros de la clase **Red_Bayesiana** es el grafo lo cual está representado como un **dicionario de python**, donde la **key** es el nodo y el **value** es la lista de adyacencia. Ejemplo:
$$\{ A: [B], B: [C], C: [] \} $$
Lo cual representa el siguiente grafo dirigido:
$$ A -> B -> C $$
A partir del grafo se genera los factores correspondientes.
Asimismo se implementó la siguiente función:
* **get_parents**: Nos devuelve un diccionario de python donde la **key** es el nodo y el **value** es un **set** de los padres de ese nodo. 

In [56]:
hide_toggle(True)

In [57]:
# Retorna los padres de los nodos
def get_parents(grafo):
    result = {}
    
    for node, list_adyacentes in grafo.items():
        if node not in result:
            result[node] = set()
        
        if (len(list_adyacentes) > 0):  
            for child in list_adyacentes:
                if child not in result:
                    result[child] = set()
                    result[child].add(node)
                else:
                    result[child].add(node)
    
    return result

class Red_Bayesiana:
    def __init__(self, df, grafo, alpha, preprocessor = None):
        self.alpha = alpha
        self.estructura = grafo
        self.parents = get_parents(self.estructura)
        self.factores = self.generate(df)
        self.card = self.get_cards(df)
        self.preprocessor = preprocessor
        
    def get_cards(self, df):
        nodes = self.estructura.keys()
        card = {}
        for value in nodes:
            card[value] = df[value].unique()
        return card
    #Genera todos los factores de acuerdo a la estructura
    def generate(self, df):
        factores = {}
        parents = self.parents
        for child, parent_list in parents.items():
            if (len(parent_list) == 0):
                factores[tuple(child)] = estimar_marginal(df,child, self.alpha)
            else:
                key = list(parent_list)
                key.insert(0, child)
                key = tuple(key)
                factores[key] = estimar_condicional(df,child, list(parent_list), self.alpha)
        return factores
    
    def printFactores(self):
        print("Factores del grafo")
        for key, df in self.factores.items():
            
            if (len(key)==1):
                print ("P(" + key[0] + ")", end=" ")
            else:
                s = None
                for i in range(len(key)):
                    if (i==0):
                        s = "P(" + key[0] + "|"
                    elif(i == len(key) - 1):
                        s = s + key[i] + ")"
                    else:
                        s = s + key[i] + ","
                print (s, end=" ")
                
    #Función para calcular todos los ancestros de un nodo
    def get_all_ancestor(self, current, current_parents, amount_of_ancestors, visited):
        visited[current] = True
        for parent in current_parents:
            if(not visited[parent]):
                self.get_all_ancestor(parent, self.parents[parent], amount_of_ancestors, visited)
            amount_of_ancestors[current] = amount_of_ancestors[current] + amount_of_ancestors[parent] + 1
        
    #Devuelve las variables ocultas ordenados en orden topológico
    def get_hidden_variables(self, consulta, evidencia):
        nodes = self.estructura.keys()
        amount_of_ancestors = {}
        visited = {}
        
        for node in nodes:
            visited[node] = False
            amount_of_ancestors[node] = 0
        
        for node in nodes:
            if(not visited[node]):
                self.get_all_ancestor(node, self.parents[node], amount_of_ancestors, visited)
        
        hidden_variables = []
        for variable in nodes:
            if ((variable not in evidencia) and variable != consulta):
                #Cada variable esta asociada a la cantidad de ancestros
                hidden_variables.append([variable, amount_of_ancestors[variable]])
        #Se ordena de menor a mayor de acuerdo a la cantidad de ancestros
        #Esto nos garantiza que el nodo que tiene padre va estar después de su padre
        hidden_variables = sorted(hidden_variables, key=lambda item: item[1])
        
        Z = []
        for i in hidden_variables:
            Z.append(i[0])
        return Z
    #El valor de la distribuicion marginal
    def estimar_marginal(self, factor, valores): 
        assert len(valores) == 1, "Marginal solo acepta un valor"
        #Se filtra el factor
        resultado = self.factores[factor]
        resultado = resultado[resultado[factor[0]]==valores[0]]
        return list(resultado['P'])[0]
    #El valor de la distribuicion condicional
    def estimar_condicional(self, factor, valores):
        #Se filtra el factor
        resultado = self.factores[factor]
        for i in range(len(factor)):
            index = factor[i]
            value = valores[i]
            #filtrar la columna 
            resultado = resultado[resultado[index] == value]
        return list(resultado['P'])[0]

## Implementación del método de inferencia
Esta función recibe como parámetros:
* La variable target o clase
* N-1 variables evidencias, es decir, esta función no está implementada para calcular inferencia con variables ocultas
* La red bayesiana

In [58]:
hide_toggle(True)

In [59]:
def inferencia(clase, e_variables , e_values, red_bayesiana):
    
    var_val = {}
    
    for i in range(len(e_variables)):
        var_val[e_variables[i]] = e_values[i]
    
    # retorna un diccionario con llaves los nodos y valores 
    parentsList = red_bayesiana.parents
    
    #Cardinalidad de la variable clase
    card = red_bayesiana.card[clase]
    list_prob = []
    
    for valor in card:
        p = 1
        var_val[clase] = valor
        for var, parents in parentsList.items():
            if (len(parents) == 0):
                p = p * red_bayesiana.estimar_marginal((var,),[var_val[var]] )
             
            else:
                
                values = [var_val[var]]
                
                for parent in parentsList[var]:
                    values.append(var_val[parent])

                key = list(parentsList[var])
                key.insert(0,var)
                key = tuple(key)
                p = p * red_bayesiana.estimar_condicional(key,values)
      
        list_prob.append(p)
    
    columns = {}
    
    columns[clase] = card
    columns['P'] = list_prob
    
    return pd.DataFrame(columns)

# Implementación del método de inferencia con variables ocultas 

Se implementó la función **get_scope** la cual nos devuelve los factores donde está incluída la variable o no.

In [60]:
hide_toggle(True)

In [61]:
def get_scope(factores, variable):
    factores_with_variable = []
    factores_without_variable = []
    for factor in factores:
        if(variable in factor):
            factores_with_variable.append(factor)
        else:
            factores_without_variable.append(factor)
    return (factores_with_variable, factores_without_variable)

def get_vars(phi_prima):
    involved = []
  
    for element in phi_prima:
        for value in element:
            if(value not in involved):
                involved.append(value)
  
    return involved

def sum_product_eliminate_var(factores, variable_to_eliminate, consulta, valor_consulta, evidencia, valor_evidencia , red_bayesiana ):
    # factores que se van a factorizar
    
    phi_prima, phi_prima_prima = get_scope(factores,variable_to_eliminate)
      
    # lista de variables involucradas en phi_prima
    variables_in_scope = get_vars(phi_prima)

    #diccionario: llave -> variable , valor -> lista de cardinalidad
    var_to_values = {}
    
    for var in variables_in_scope:
        if (var != consulta and (var not in evidencia)):
            var_to_values[var] = red_bayesiana.card[var]
    var_to_values[consulta] = [valor_consulta]
    
    for i in range(len(evidencia)):
        var_to_values[evidencia[i]] = [valor_evidencia[i]]
    
    # llave: variable , valor --> indice en una n_upla
    var_to_index = {}

    #total number of posibilites
    n =  1
    for var, val in var_to_values.items():
        n = n*len(val)
  
    posibilidades =  []
    for i in range(n):
        posibilidades.append([])
        
    groups = 1
    pos = 0
    for var in variables_in_scope:
        var_to_index[var] = pos
        pos = pos + 1
        values = var_to_values[var]
        m = len(values)
        groups = groups*m
        rep = int(n/groups)
        for i in range(groups):
            for j in range(rep):
                posibilidades[i*rep + j].extend([values[i%m]])
    # factores que quedan
  
    p = 0 
    # iterar entre todas las posibilidades
    for n_upla in posibilidades:
    
        p_local = 1
        for dist in phi_prima:
            var = dist[0] # El nodo
            cond = dist[1:] # Los padres del nodo
            
            # hallar los valores que tomas las variables en la distribuiciones
            if (len(cond) == 0):
                p_marginal = red_bayesiana.estimar_marginal(dist,[n_upla[var_to_index[var]]])
                p_local = p_local*p_marginal
            else:
                valores = []
                for var in dist:
                    index = var_to_index[var]
                    valores.append(n_upla[index])
                
                p_condicional = red_bayesiana.estimar_condicional(dist, valores)
              
                p_local = p_local*p_condicional
        # sumatoria de los valores en los factores dentro del get_scope
        p = p + p_local
        
    return [phi_prima_prima, p]

def sum_product(factores, consulta, valor_consulta, evidencia, valor_evidencia, red_bayesiana):
    
    var_to_val = {}
    for i in range(len(evidencia)):
        var_to_val[evidencia[i]] = valor_evidencia[i]
    
    var_to_val[consulta] = valor_consulta
    
    p = 1
    
    for dist in factores:
        var = dist[0]
        cond = dist[1:]
        if (len(cond) == 0):
            p_marginal = red_bayesiana.estimar_marginal(dist,[var_to_val[var]])
            p = p*p_marginal
        else:
            valores = []
            for var in dist:
                valores.append(var_to_val[var])
         

            p_condicional = red_bayesiana.estimar_condicional(dist, valores)
           
            p = p*p_condicional
    
    return p

def sum_product_ve(consulta, valor_consulta , evidencia, valor_evidencia, red_bayesiana):
    Z = red_bayesiana.get_hidden_variables(consulta, evidencia)
  
    tetha = []
    factores = red_bayesiana.factores.keys() # lista de factores
    for i in range(len(Z)):
  
        
        if(len(factores) == 0):
            break
        [nuevos_factores,sumatoria] =  sum_product_eliminate_var(factores, Z[i], consulta, 
                                                                 valor_consulta, evidencia, 
                                                                 valor_evidencia, red_bayesiana)
    
        factores = nuevos_factores
        tetha.append(sumatoria)
    
    if(len(factores)>0):
        p_last = sum_product(factores,consulta,  valor_consulta, evidencia,valor_evidencia, red_bayesiana)
        tetha.append(p_last)
    p = 1
    for sumatoria in tetha:
        p = p*sumatoria
    
    return p

def inferencia_con_variables_ocultas(consulta, evidencia, valor_evidencia, red_bayesiana):
    
    card = red_bayesiana.card[consulta]
    list_prob = []
    for valor_consulta in card:
        p = sum_product_ve(consulta, valor_consulta , evidencia, valor_evidencia, red_bayesiana)
        list_prob.append(p)
    columns = {}
    columns[consulta] = card
    columns['P'] = list_prob
    
    return pd.DataFrame(columns)

## Definición e implementación de la clase Structure
Esta clase representa la estructura de la red bayesiana, es decir, el grafo dirigido y acíclico (DAG). Asimismo, para verificar si existe ciclos en el grafo se implementó la función que chequea ello **is_cyclic**.

También, se implementó la función **get_graph_id** la cual devuelve un identificador al grafo. Este identificador es de acuerdo a las aristas dirigidas que están presente. Ejemplo:

Sea el grafo con los siguientes vertices: 
$$ V = \{A, B, C\} $$ 
y las aristas:
$$ E = \{A->B, C->A\}$$
Por lo tanto el identificador de este grafo es:

|    | AA | AB | AC | BA | BB | BC | CA | CB | CC |
|----|:--:|----|----|----|----|----|----|----|----|
| ID | 0  | 1  | 0  | 0  | 0  | 0  | 1  | 0  | 0  |

Asimismo, se implementaron las siguientes funciones:
* **operator_for_eliminate_edge**: Elimina cualquier arista.
* **operator_for_add_edge**: Agrega una arista válida. La arista $A->A$ se considera inválida.
* **operator_for_reverse_edge**: Invierte una arista si es posible.

In [62]:
hide_toggle(True)

In [63]:
class Structure:
    def __init__(self, list_adj ,preprocessor = None):
        self.estructura = copy.deepcopy(list_adj)
        self.parents = get_parents(self.estructura)
        self.set_edge = self.get_set_edge(list_adj)
        self.n_nodes = len(list_adj.keys())
        self.n_edges = len(self.set_edge)
        self.nodes_indexes = self.get_indexes()
        self.graph_id = self.get_graph_id()
        self.edges_not_graph = self.get_edges_not_graph()   
        self.preprocessor = preprocessor
    
    def get_indexes(self):
        indexes = {}
        i = 0
        for node in self.estructura.keys():
            indexes[node] = i
            i = i + 1
        return indexes
    
    def get_graph_id(self):
        VxV = self.n_nodes * self.n_nodes
        g_id = [0] * (VxV)
        for node, list_adj in self.estructura.items():
            for neighbor in list_adj:
                index = self.n_nodes * self.nodes_indexes[node] + self.nodes_indexes[neighbor] 
                g_id[index] = 1
        return g_id

    def get_set_edge(self, list_adj):
        result = set()
        for v, list_neighbors in list_adj.items():
            for neighbor in list_neighbors:
                result.add((v,neighbor))
        return result

    def get_edges_not_graph(self):
        edges_possibles = set()
        i = 0
        j = self.n_nodes - 1
        k = 0
        nodes = list(self.nodes_indexes.keys())
        for index in range(len(self.graph_id)):
            if(index != i):
                if(self.graph_id[index] == 0):
                    edges_possibles.add((nodes[i%self.n_nodes], nodes[k%self.n_nodes]))
            if(j == index):
                i = i + self.n_nodes + 1
                j = j + self.n_nodes
                
            k = k + 1
            
        return edges_possibles
    
    def add_edge(self,v1,v2):
        edge = (v1, v2)
        self.edges_not_graph.remove(edge)
        self.set_edge.add(edge)
        
        self.estructura[v1].append(v2)
        self.parents[v2].add(v1)
        self.n_edges = self.n_edges + 1
        
        index = self.n_nodes*self.nodes_indexes[v1] + self.nodes_indexes[v2]
        self.graph_id[index] = 1 # this means that a edge was added
    
    def operator_for_eliminate_edge(self):
        if (self.n_edges == 0):
            return
        edge = random.choice(list(self.set_edge))
        self.set_edge.remove(edge)
        self.edges_not_graph.add(edge)
        
        v1 = edge[0]
        v2 = edge[1]
        self.estructura[v1].remove(v2)
        self.parents[v2].remove(v1)
        self.n_edges = self.n_edges - 1
        
        #Update graph id
        index = self.n_nodes*self.nodes_indexes[v1] + self.nodes_indexes[v2]
        self.graph_id[index] = 0 # this means that a edge was removed
        return edge
    
    def operator_for_add_edge(self):
        total_edges = (self.n_nodes) * (self.n_nodes - 1)
        if(total_edges == len(self.set_edge)):
            return
        
        edge = random.choice(list(self.edges_not_graph))
        self.edges_not_graph.remove(edge)
        self.set_edge.add(edge)
        v1 = edge[0] 
        v2 = edge[1]
        self.estructura[v1].append(v2)
        self.parents[v2].add(v1)
        
        index = self.n_nodes*self.nodes_indexes[v1] + self.nodes_indexes[v2]
        self.graph_id[index] = 1 # this means that a edge was added
        
        self.n_edges = self.n_edges + 1
        
    def operator_for_reverse_edge(self):
        total_edges = (self.n_nodes) * (self.n_nodes - 1)
        if (self.n_edges == 0 or self.n_edges == total_edges):
            return
        
        possible = False
        inverted_edge = None
        edge_to_remove = None
        for graph_edge in self.set_edge:
            inverted = (graph_edge[1], graph_edge[0])
            if (not (inverted in self.set_edge)):
                edge_to_remove = graph_edge
                inverted_edge = inverted
                possible = True
                break
        
        if(not possible):
            return
        
        self.set_edge.remove(edge_to_remove)
        self.edges_not_graph.add(edge_to_remove)
        
        v1 = edge_to_remove[0]
        v2 = edge_to_remove[1]
        self.estructura[v1].remove(v2)
        self.parents[v2].remove(v1)
        
        v1 = inverted_edge[0]
        v2 = inverted_edge[1]
        self.estructura[v1].append(v2)
        self.parents[v2].add(v1)

        self.set_edge.add(inverted_edge)
        self.edges_not_graph.remove(inverted_edge)
        
        index = self.n_nodes*self.nodes_indexes[v1] + self.nodes_indexes[v2]
        self.graph_id[index] = 1 # this means that a edge was added
        index = self.n_nodes*self.nodes_indexes[v2] + self.nodes_indexes[v1]
        self.graph_id[index] = 0 # this means that a edge was removed
        
    def is_cyclic_util(self, v, visited, recStack):
        visited[v] = True
        recStack[v] = True
        
        for neighbour in self.estructura[v]:
            if(not visited[neighbour]):
                if (self.is_cyclic_util(neighbour, visited, recStack)):
                    return True
            elif (recStack[neighbour]):
                return True
        
        recStack[v] = False
        return False
    
    def is_cyclic(self):
        nodes = self.estructura.keys()
        visited = {}
        recStack = {}
        for node in nodes:
            visited[node] = False
            recStack[node] = False

        for node in nodes:
            if (not visited[node]):
                if self.is_cyclic_util(node, visited, recStack):
                    return True
        
        return False

# Métricas de calidad
Para poder asignar un puntaje a la estructura de la red bayesiana se define las siguientes métricas:
## Métrica de entropía

In [64]:
hide_toggle(True)

In [65]:
def get_entropia(estructura_red):
    ## Obtenemos la cardinalidad de cada variable
    variables = estructura_red.estructura.keys()
    n = len(variables)
    # diccionario de variable -> lista de valores

    r = {} 
    for var in variables:
        r[var] = estructura_red.preprocessor.card[var]

    
    # diccionario de variables -> lista de padres
    q = estructura_red.parents
    #TO DO
    
    # N total de instancias
    N = estructura_red.preprocessor.instancias
    result = 0
    
    # i = 1 ... n
    # xi
    for variable in variables:
        xi = variable
        pa_xi = list(q[variable])
        
        # si la variable no tiene padre
        if(len(pa_xi)==0):
            for valor in r[variable]:
                
                # Ej: P(A = 0).log2(P(A=0))
                prob_xi = estructura_red.preprocessor.get_preprocess_marginal(xi, valor)
                result = result + prob_xi*math.log(prob_xi,2)
            
            continue
        
        # j = 1 .... qi
        pa_xi = pa_xi[0]
        for parent_val in r[pa_xi]:
            # k = 1... ri
            for valor in r[variable]:                
                # Calculo de P(xi|pa(xi))
                p_xi_paxi = estructura_red.preprocessor.get_preprocess_condicional((xi,pa_xi),[valor,parent_val])
                p_conj_xi_pxi = estructura_red.preprocessor.get_preprocess_conjunta((xi,pa_xi),[valor,parent_val])
     
                result = result + p_conj_xi_pxi*(math.log(p_xi_paxi,2))
        
    result = result*(-1)*N
                
    return result

## Métrica de akaike

In [66]:
hide_toggle(True)

In [67]:
def get_number_independent_parameters(estructura_red):
    cardinalities = estructura_red.preprocessor.card
    parents = estructura_red.parents
    
    sum = 0
  
    for r in cardinalities.keys():
        q = 1
        for parent in parents[r]:
            q = q * len(cardinalities[parent])
        sum = sum + (len(cardinalities[r]) - 1)*q

    return sum

def get_akaike(estructura_red):
    #
    K = get_number_independent_parameters(estructura_red)
    e = get_entropia(estructura_red)
    
    return e + K

## Métrica de MDL

In [68]:
hide_toggle(True)

In [69]:
def get_mdl(estructura_red):    
    N = estructura_red.preprocessor.instancias
    K = get_number_independent_parameters(estructura_red)
    e = get_entropia(estructura_red)
    
    return e + K/2*math.log(N,2)

##  Métrica K2

In [70]:
hide_toggle(True)

In [71]:
def get_k2(estructura_red):
    # Obtenemos la cardinalidad de cada variable
    variables = estructura_red.preprocessor.variables
    n = len(variables)
    df = estructura_red.preprocessor.df
    q = estructura_red.parents
    
    r = {} 
    for var in variables:
        r[var] = estructura_red.preprocessor.card[var]
    
    result = 1
    
    for variable in variables:
        xi = variable
        pa_xi = list(q[variable])
        if(len(pa_xi)==0):
            continue
        
        ri = len(r[variable])
        
        # j = 1 .... qi
        pa_xi = pa_xi[0]
        for parent_val in r[pa_xi]:
            
            # Nij número de instancias donde Pa(xi) toma su j-esimo valor 
            Nij = count(df, [pa_xi], [parent_val])
            
            
            a = math.factorial(ri -1) / math.factorial(ri-1 + Nij)
            
            result = result*a
            # k = 1... ri
            for valor in r[variable]:
                
                # Nijk número de instancias donde Pa(xi) toma su j-esimo valor y xi toma su k-esimo valor
                Nijk = count(df, [pa_xi,xi],[parent_val,valor])
                result =  result* math.factorial(Nijk)
    
    return result

## Greedy local search algorithm
Las funciones previamente implementadas sirven como heurísticas para evaluar la calidad de diferentes estructuras de red bayesiana.

Por lo que podemos definir una función para encontrar una estructura que obtiene el máximo score de acuerdo a la función de métrica que le pasemos

In [72]:
hide_toggle(True)

In [73]:
# set_operators [0, 1, 2]: 0 = add, 1 = remove, 2 = revert
def greedy_local_search(initial_solution, score_function, set_operators = [0, 1, 2]):
    best_solution = initial_solution
    Progress = True
    while(Progress):
        solution = best_solution
        Progress = False
        for operator in set_operators:
            
            candidate_solution = copy.deepcopy(solution)
            if(operator == 0):
                candidate_solution.operator_for_add_edge()
            elif(operator == 1):
                candidate_solution.operator_for_eliminate_edge()
            else:
                candidate_solution.operator_for_reverse_edge()
            
            if (not candidate_solution.is_cyclic()):
                if(score_function(candidate_solution) > score_function(best_solution)):
                    best_solution = copy.deepcopy(candidate_solution)
                    Progress = True
    return best_solution

## K2 search algorithm

In [74]:
hide_toggle(True)

In [75]:
## se pasa una estructura que no tiene aristas unidas 
def k2_algorithm( initial_solution,score_function, u ):
    # ordenar de forma topológica
    variables  = initial_solution.preprocessor.variables
    
    #seteamos vacia al conjunto de padres de cada variable
    padres = {}
    for var in variables:
        padres[var] = []
    
    best_solution = initial_solution
    
    for i in range(len(variables)):
        var = variables[i]
        p_old = score_function(best_solution)
        okToProceed = True
        
        while (okToProceed and  len(padres[var]) < u):
            ##hallar  pred(x) - pa(x) que maximiza 
            pred = variables[0:i]
            
            #candidate_solution = copy.deepcopy(solution)
            
            z = []
            for v in pred:
                if (v not in padres[var]):
                    z.append(v)
            
            # hallar el nodo en z que maximiza la funcion 
            
            
            p_new = 0
            z_max = None
            for node in z:
                candidate_solution = copy.deepcopy(best_solution)
                candidate_solution.add_edge(node,var)
                p_temp = score_function(candidate_solution)
                if(p_temp > p_new):
                    p_new = p_temp
                    z_max = node
                # simular que node es padre de var y hallar el mayor
                
                
            if (p_new > p_old):

                p_old = p_new
                # añadimos a z como padre
                best_solution.add_edge(z_max, var)
                padres[var].append(z_max)
                # actualizamos la estructura de la solución
                
                
            else:
                okToProceed = False
    
    return best_solution

<a id='presentacion'></a>
# Presentación del proyecto en general


Este proyecto tiene como objetivo buscar la mejor estructura que modele los datos de una base de datos, que se usará posteriormente para realizar inferencias.

Comenzaremos cargando la base de datos de estudio que también es llamada base de datos de entrenamiento.

In [76]:
hide_toggle(True)

In [77]:
import ipywidgets as widgets
from IPython.display import display

uploader = widgets.FileUpload(
    accept= '.csv', 
    multiple=False  
)
button = widgets.Button(description="Read .csv!")
out = widgets.Output()
display(uploader, button, out)

df_train = None # Dataframe de entrenamiento
path = None

def load_csv(extra):
    global path, df_train
    for name, file_info in uploader.value.items():
        path = name 
        with open (name, 'wb') as file:
            file.write(file_info['content'])
    df_train = leer_dataset(path)
    with out:
        display(df_train)
        
button.on_click(load_csv)

FileUpload(value={}, accept='.csv', description='Upload')

Button(description='Read .csv!', style=ButtonStyle())

Output()

Luego procedemos a leer el grafo, el cual en el archivo de texto debería de estar de la siguiente forma para que sea válido el procesamiento. Este **es un ejemplo**:
$$\{ 'A': [], 'B': ['A'], 'C': [] \}$$
Asimismo, a partir del grafo se establece una hipotesis de cómo están relacionadas estas variables. La hipotesis se modelará como un DAG (directed aciclyc graph) donde un vertice representa una variable  y una arista (v1,v2) representa que la variable v2 depende de la variable v1.

In [78]:
hide_toggle("True")

In [79]:
import ast
from graphviz import Digraph
uploader = widgets.FileUpload(
    accept= '.txt', 
    multiple=False  
)
button = widgets.Button(description="Read graph!")
out2 = widgets.Output()
display(uploader, button, out2)

test_graph = None # Dataframe de entrenamiento

def to_visual(graph):
    dot = Digraph(format='png')
    for vertice in graph.keys():
        dot.node(str(vertice))
    for node, list_adj in graph.items():
        for neighbour in list_adj:
            edge = str(node)+str(neighbour)
            dot.edges([edge])
    return dot

def read_graph(extra):
    global test_graph
    for name, file_info in uploader.value.items():
        path = name 
        with open (name, 'wb') as file:
            file.write(file_info['content'])
        f = open('prueba.txt')
        test_graph = ast.literal_eval(f.read())
        graph = to_visual(test_graph)
    with out2:
        display(graph)
        
button.on_click(read_graph)

FileUpload(value={}, accept='.txt', description='Upload')

Button(description='Read graph!', style=ButtonStyle())

Output()

Este estudio está basado en redes bayesianas y se calcula las probabilidades usando el parametro de Dirichlet escrito normalmente como alpha

In [80]:
hide_toggle(True)

In [81]:
alpha = 1.0 #SET ALPHA

Antes de continuar se realizará un preprocesamiento de los datos. Esto mejora el tiempo de respuesta al calcular las metricas de calidad de 

A continuación se mostrará los factores de la red bayesiana

In [82]:
hide_toggle(True)

In [83]:
red = Red_Bayesiana(df = df_train, grafo = test_graph, alpha=alpha )
red.printFactores()

AttributeError: 'NoneType' object has no attribute 'items'

A continuación se muestra el histograma de probabilidades para cada variable.

In [ ]:
hide_toggle(True)

In [ ]:
import matplotlib.pyplot as plt
def generate_histogram(df):
    variables = []
    for i in df:
        variables.append(i)
    var = variables[0]
   
    x = []
    y = []
    
    for i in df.index:

        x.append(str(df[var][i]))
        y.append(df['P'][i])
    
    fig, ax = plt.subplots()
    ax.bar(x,y)
    plt.show()
    
for var in df_train:
    print("Variable ",var)
    marginal = estimar_marginal(df_train,var, alpha)
    generate_histogram(marginal)

Ahora veremos la distribución conjunta de todas las variables del dataset

In [ ]:
hide_toggle(True)

In [ ]:
estimar_conjunta(df_train, ['A','B','C','D','E','F'], alpha=alpha)

## Inferencia

Hay 2 tipos de consultas de inferencia. La primera es teniendo las variables evidencia y consulta. Mientras que la segunda es cuando las variables evidencia no cubren todas las variables restantes, a las variables que no están en la evidencia y la consulta se les denomina variables aculta.

### Inferencia con evidencia y consulta

Aqui se calcula la probabilidad de la variable consulta a partir de la evidencia

Como ejemplo, sumpongamos que queremos consultar la variable G sabiendo que tenemos como evidencia a:

A = 1, B = 1, C = 2, D = 0, E = 1, F = 3

Con esta información veamos la probabilidad de G para cada uno de sus valores.

In [ ]:
hide_toggle(True)

In [ ]:
inferencia('G',['A','B','C','D','E','F'],[1,1,2,0,1,3], red)

Observamos que P(G=0) es el mayor valor por lo que tiene mayor probabilidad de ser correcto.

### Inferencia con variables ocultas

Aqui se calcula la probabilidad de la variable consulta a partir de la evidencia y las variables ocultas

Como ejemplo, sumpongamos que queremos consultar la variable G sabiendo que tenemos como evidencia a:

A = 1, B = 1, C = 2

Entonces las variables ocultas serian:

D, E y F.


In [ ]:
hide_toggle(True)

In [ ]:
inferencia_con_variables_ocultas('G', ['A','B','C'],[1,1,2], red)

En este caso también observamos que P(G=1) aunque con menor intensidad que el caso anterior.

## Busqueda de la mejor estructura

Esta sección tiene como objetivo buscar la estructura (DAG) que mejor se adapte a los datos que tenemos. Esta busqueda se realizará con 2 algoritmos: greedy search y k2 algorithm. Estos algoritmos usan metricas de calidad para comparar 2 estructuras diferentes. A continuación veremos las 4 métricas implementadas.

Antes de proceder con las métricas, se realizará un preprocesamiento para mejorar el tiempo de respuesta en el calculo de métricas de calidad, en consecuencia también se mejorará el tiempo de respuesta en los algoritmos de busqueda.




### Preprocesamiento

Se calcula las probabilidades conjuntas de cada par de variables, tambien las probabilidades marginales para cada variable y las probabilidades condicionales para cada par de variables. Y también vamos a convertir el grafo en una estrucura que es una clase que contiene más funciones como añadir vertices, eliminar vertices, etc.


In [ ]:
hide_toggle(True)

In [ ]:
preprocessor = Preprocessor(df=df_train, alpha=alpha)
test_graph = Structure(test_graph, preprocessor=preprocessor)

### Métrica de entropía

In [ ]:
hide_toggle(True)

In [ ]:
get_entropia(test_graph)

### Métrica de akaike

In [ ]:
hide_toggle(True)

In [ ]:
get_akaike(test_graph)

### Métrica de MDL

In [ ]:
hide_toggle(True)

In [ ]:
get_mdl(test_graph)

### Métrica K2

In [ ]:
get_k2(test_graph)

 Exploraremos el 25 % del espacio total que es aproximadamente $$(2\^(42))*0.25$$

In [ ]:
hide_toggle(True)

In [ ]:
def greedy_local_search_modified(initial_solution, score_function, set_operators = [0, 1, 2]):
    best_solution = initial_solution
    Progress = True
    iteraciones = 0
    while(Progress):
        solution = best_solution
        Progress = False
        for operator in set_operators:
            candidate_solution = copy.deepcopy(solution)
            if(operator == 0):
                candidate_solution.operator_for_add_edge()
            elif(operator == 1):
                candidate_solution.operator_for_eliminate_edge()
            else:
                candidate_solution.operator_for_reverse_edge()
            
            iteraciones = iteraciones + 1
            if (not candidate_solution.is_cyclic()):
                if(score_function(candidate_solution) > score_function(best_solution)):
                    best_solution = copy.deepcopy(candidate_solution)
                    Progress = True
    return (best_solution, iteraciones)

def greedy_algorithm_search_space(initial_graph, score_function):
    N = initial_graph.n_nodes
    MAX_ITERACIONES = 2**(N*(N-1)) * 0.25
    print(MAX_ITERACIONES)
    MAX = 5
    graph = copy.deepcopy(initial_graph)
    g_id = initial_graph.graph_id
    same_id = 0
    current_iteration = 0
    while(current_iteration < MAX_ITERACIONES):
        while(current_iteration < MAX_ITERACIONES and same_id < MAX):
            graph, iteraciones = greedy_local_search_modified(graph, score_function, [0, 1, 2])
            current_iteration = current_iteration + iteraciones
            print("*******************")
            print(graph.estructura)
            print(current_iteration)
            print("*******************")
            if(graph.graph_id == g_id):
                same_id = same_id + 1
            else:
                g_id = graph.graph_id
                same_id = 0
        
        if(current_iteration < MAX_ITERACIONES):
            new_graph = graph
            candidate_graph = copy.deepcopy(new_graph)
            cant = 3
            op = random.randint(0,2)
            if(op == 0):
                for i in range(cant):
                    candidate_graph.operator_for_add_edge()
                    if(not candidate_graph.is_cyclic()):
                        new_graph = candidate_graph
                    candidate_graph = copy.deepcopy(new_graph)
                        
            elif(op == 1):
                for i in range(cant):
                    graph.operator_for_eliminate_edge()
                    if(not candidate_graph.is_cyclic()):
                        new_graph = candidate_graph
                    candidate_graph = copy.deepcopy(new_graph)
            else:
                for i in range(cant):
                    graph.operator_for_reverse_edge()
                    if(not candidate_graph.is_cyclic()):
                        new_graph = candidate_graph
                    candidate_graph = copy.deepcopy(new_graph)
            print("nuevo grafo")
            print(graph.estructura)
            same_id = 0
            current_iteration = current_iteration + cant
            graph = new_graph

            
    return current_iteration

initial_graph = {'A':[],'B': [], 'C':[], 'D':[],'E':[],'F':[],'G':[]}
initial_graph = Structure(initial_graph, preprocessor)

greedy_algorithm_search_space(initial_graph, get_akaike)

Ahora pongamos en acción los algoritmos de busqueda de la mejo0r estructura

### Greedy Search

In [ ]:
hide_toggle(True)

In [ ]:
initial_graph = {'A':[],'B': [], 'C':[], 'D':[],'E':[],'F':[],'G':[]}
initial_graph = Structure(initial_graph, preprocessor)

nuevo_grafo = greedy_local_search_modified(initial_graph, get_akaike)
print("La mejor solución es :")
print(nuevo_grafo[0].estructura)
print("Se generaron: ",nuevo_grafo[1])
display(to_visual(nuevo_grafo[0].estructura))

### Algoritmo K2

In [ ]:
hide_toggle(True)

In [ ]:

initial_graph = {'A':[],'B': [], 'C':[], 'D':[],'E':[],'F':[],'G':[]}
initial_graph = Structure(initial_graph, preprocessor)

nuevo_grafo = k2_algorithm(initial_graph, get_akaike, 1)
print("La mejor solución es :")
print(nuevo_grafo.estructura)
display(to_visual(nuevo_grafo.estructura))
    